In [1]:
import pandas as pd
import numpy as np

# static lists to get players and teams
from nba_api.stats.static import players
from nba_api.stats.static import teams

# endpoints to get the list of players on every team and shot chart for each player
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import teamplayerdashboard

# getting nba defensive statistics
from nba_api.stats.endpoints import boxscoredefensive
from nba_api.stats.endpoints import defensehub
from nba_api.stats.endpoints import leaguedashptdefend
from nba_api.stats.endpoints import playerdashptshotdefend


# getting nba player info for their position
from nba_api.stats.endpoints import commonplayerinfo

## Finding all the nba shot for each player from each game for the 2019/20 nba season

In [2]:
# importing a dictionary of the nba teams to get a list of the team ids

teams_dict = teams.get_teams()

In [3]:
# obtaining list of team ideas for each team in the team dictionary

team_ids = [team['id'] for team in teams_dict]

In [23]:
df_list = []

In [31]:
# looping through each team id in the list of team ids
for team in team_ids:
    # obtaining the dashboard of the teams with all the active players and their stats
    team_players = teamplayerdashboard.TeamPlayerDashboard(team, season = '2019-20')

    # using that list to grab the player ids from every player on every team
    player_ids = team_players.players_season_totals.get_data_frame().PLAYER_ID

    # looping through all the players on every team
    for player_id in player_ids:

        # getting the shot chart object from the nba api for each player using the team id and the player id
        shot_chart = shotchartdetail.ShotChartDetail(team, player_id)

        # turning that shot chart object into a dataframe
        shot_chart_df = shot_chart.get_data_frames()[0]

        # changing the dates into a datetime to grab the games from 2019-2020, then setting that as the index
        shot_chart_df['GAME_DATE'] = pd.to_datetime(shot_chart_df['GAME_DATE'], format = '%Y%m%d', errors = 'ignore')
        shot_chart_df.set_index('GAME_DATE', inplace = True)

        # locating only games for the 2019/20 nba season
        shot_chart_2020 = shot_chart_df.loc['2019-10-22':'2020-03-11']

        # appending it to a list to later concat into a single DataFrame
        df_list.append(shot_chart_2020)

In [34]:
# concating the list of dataframes into a single dataframe

df = pd.concat(df_list)

In [38]:
# storing that in a csv

df.to_csv('2019-20_nba_shot_charts.csv')

## Finding all shot charts for all players from 1999-00 season to present

In [24]:
# getting a list of all the nba seasons from 1999-00 season to present

seasons = ['2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14',
           '2012-13', '2011-12', '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', 
           '2005-06', '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00']

In [25]:
# list to append all the dataframes of player shot charts to
all_seasons_list = []

# list to store player ids so the same player isnt used more than once
used_players = []


# looping through each team id in the list of team ids
for team in team_ids:
    
    # looping through each season to grab the players from the team in that season
    for season in seasons:
        
        try:
            # obtaining the dashboard of the teams with all the active players and their stats
            team_players = teamplayerdashboard.TeamPlayerDashboard(team, season = season)

            # using that list to grab the player ids from every player on every team
            player_ids = team_players.players_season_totals.get_data_frame().PLAYER_ID
            
            # looping through all the players on every team
            for player_id in player_ids:
                
                if player_id not in used_players:
                    
                    # getting the shot chart object from nba api for each player using team id and player id
                    shot_chart = shotchartdetail.ShotChartDetail(team, player_id)

                    # turning that shot chart object into a dataframe
                    shot_chart_df = shot_chart.get_data_frames()[0]

                    # changing the dates into a datetime to grab the games from 2019-2020, then setting that as the index
                    shot_chart_df['GAME_DATE'] = pd.to_datetime(shot_chart_df['GAME_DATE'], format = '%Y%m%d', errors = 'ignore')
                    shot_chart_df.set_index('GAME_DATE', inplace = True)

                    # locating only games for the 2019/20 nba season
                    shot_chart_all = shot_chart_df.loc['1999-11-02':'2020-03-11']
                    
                    # appending it to a list to later concat into a single DataFrame
                    all_seasons_list.append(shot_chart_all)
                    
                    # appending the player id to store and not use again in case it comes up for another team
                    used_players.append(player_id)
                    
                else:
                    pass
                
        except JSONDecodeError:
            pass

In [26]:
all_seasons = pd.concat(all_seasons_list)

In [46]:
all_seasons.to_csv('2000-2020_shot_charts.csv')

## Finding nba defensive statistics

In [44]:
# Getting a dictionary of all the player info

players_dict = players.get_active_players()

In [45]:
# getting a list of all player ids

player_ids = [player['id'] for player in players_dict]

In [65]:
df_each_game = df.drop_duplicates('GAME_ID')

In [71]:
boxscore_list = []

In [72]:
# getting all defensive box score stats for each game

# looping through each unique game in the 2019/20 season
for game in df_each_game.GAME_ID:
    
    # getting the boxscore object from the nba api and converting it into a dataframe
    boxscore = boxscoredefensive.BoxScoreDefensive(game)
    boxscore_df = boxscore.get_data_frames()[0]
    
    # appending the box score to the empty list to eventually concat into a df
    boxscore_list.append(boxscore_df)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
boxscore_df = pd.concat(boxscore_list)

In [ ]:
len(boxscore)